In [ ]:
import talib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from binance.client import Client
from binance import BinanceSocketManager
import sqlalchemy
import csv
import os

In [ ]:
api_key = os.environ.get('binance_api')
api_secret = os.environ.get('binance_secret')
api_key, api_secret = ('1WsWdt8Lv5tlW0Z30of7ju97blJl95SalTwvGWSmzBxE8soy75fbuyc3jt36SAuQ', '5mCioFNLgxIs78ia2Ni5p8wke9nJXzk6RRcYeW9n6rO6Zr5oE6Tu6LLJqJZBy7Bj')
api_key_demo, api_secret_demo = ('MC4hgBp5RK6sBvwPDhMfPkcKgnFxnzsz68ppKZGRsEmQj5gRml9tiMANQBW5eqKp', 'ZvXbnaTy0F7bpZKby2USianD9mNYFSmmPWrKje4FntCeMxKlh8NNRjU9qgUGYalH')
client = Client(api_key, api_secret)

In [ ]:
# AAroon strategy Parameters
# core mean first trade!
aroon_time_period = 3
core_to_trade = True 
core_quantity = 0
amount = 1000
core_trade_amount = amount*0.80  # first trade 80% 
trade_amount = amount*0.20
money_end = amount
portfolio = 0 # numbr of BTC we own as a portfolio
investment = [] # trading has been executed array
real_time_portfolio_value = []

In [ ]:
# trading strategy function implementation
# if statement below : if i inveted 800$ (last invested amount)in BTC and i need 100$ (new inv amount)in BTC, it will add 100 to 800 total 900

def buy(allocalted_money, price):
    global money_end, portfolio, investment
    quantity = allocalted_money / price 
    money_end -= quantity*price # money end will substraction 
    portfolio += quantity # portfolio will increase
    if investment == []: # investment tracking
        investment.append(allocalted_money)
    else:
        investment.append(allocalted_money)
        investment[-1] += investment[-2]

def sell(allocalted_money, price):
    global money_end, portfolio, investment
    quantity = allocalted_money / price
    money_end += quantity*price # adding because selling
    portfolio -= quantity
    investment.append(-allocalted_money) # when selling 
    investment[-1] += investment[-2] # remaining investment 


In [10]:
cc = 'btcusd'
interval = '1m'
socket = f'wss://stream.binance.com:9443/ws/{cc}t@kline_{interval}'

In [11]:
import websocket, json, pprint
import _thread
import time

closes = []
highs = []
lows = []

def on_close(ws):
    global closes, highs, lows, core_to_trade, core_quantity, money_end, portfolio, investment, core_to_trade, last_price

    portfolio_value = portfolio*closes[-1]
    if portfolio_value > 0:
        sell(portfolio_value, price = closes[-1])
    else:
        buy(-portfolio_value, price = closes[-1])
    money_end += investment[-1] 
    print("All rest Trades stays")


def on_message(ws, message):
    global closes, highs, lows, core_to_trade, core_quantity, money_end, portfolio, investment, core_to_trade, last_price
    json_message = json.loads(message)
    cs = json_message['k'] # candle stick
    candle_closed, close, high, low = cs['x'], cs['c'], cs['h'], cs['l']

    if candle_closed:
        closes.append(float(close))
        highs.append(float(high))
        lows.append(float(low))
        last_price = closes[-1]
        print(f'Closes: {closes}')

        if core_to_trade:
            buy(core_trade_amount, price = closes[-1])
            print(f'Core Investment: we bought ${core_trade_amount} worth of bitcoin', '\n')
            core_quantity += core_trade_amount / closes[-1]
            core_to_trade = False
    aroon = talib.AROONOSC(np.array(highs), np.array(lows), aroon_time_period)
    # we need the last aaroon value
    last_aroon = round(aroon[-1], 2)
    print(last_aroon) # take the last aroon value to decide the remaining 200$ where to invest
    amt = last_aroon * trade_amount / 100 # if the last aroon is 66.67 * 200 which is 133.34 from total 200 can we invest the second as buy
    portf_value = portfolio*last_price - core_quantity*last_price # net portfolio
    trade_amt = amt - portf_value

    RT_portfolio_value  = portf_value + core_quantity*last_price + money_end
    real_time_portfolio_value.append(float(RT_portfolio_value))


    print(f'The last aroon is "{last_aroon}" and recommended exposure is "${amt}"')
    print(f'The real time portfolio: ${RT_portfolio_value}', '\n')

    if trade_amt > 0:
        buy(trade_amt, price=last_price)
        print(f'we bought ${trade_amt} worth of bitcoin', '\n', '\n')
    elif trade_amt < 0:
        sell(-trade_amt, price=last_price)
        print(f'we sold ${trade_amt} worth of bitcoin', '\n', '\n')





ws = websocket.WebSocketApp(socket, on_message = on_message, on_close = on_close)



In [12]:
ws.run_forever()

False

In [14]:
beg = closes[0]
end = closes[-1]

btc_return = (end/beg-1)
btc_return*100

IndexError: list index out of range

In [9]:
bot_return =(money_end/amount -1)
bot_return*100

NameError: name 'money_end' is not defined

In [ ]:
alpha = bot_return - btc_return
alpha*100

In [ ]:
btc_risk = np.std(np.log(np.array(closes[1:]) / np.array(closes[:-1])))
btc_risk

In [ ]:
bot_risk = np.std(np.log(np.array(real_time_portfolio_value[1:]) / np.array(real_time_portfolio_value[:-1])))
bot_risk

In [ ]:
bot_sharp_ratio = bot_return / bot_risk
round(bot_sharp_ratio,3)

In [1]:
list = []
numbers = 100
for i in range(numbers):
    list.append(i - 1)
    print(list)
    


[-1]
[-1, 0]
[-1, 0, 1]
[-1, 0, 1, 2]
[-1, 0, 1, 2, 3]
[-1, 0, 1, 2, 3, 4]
[-1, 0, 1, 2, 3, 4, 5]
[-1, 0, 1, 2, 3, 4, 5, 6]
[-1, 0, 1, 2, 3, 4, 5, 6, 7]
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8]
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1